In [1]:
import pickle
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from src.models import rnn, lstm, bjrnn, cfrnn, copulaCPTS, dplstm, vanila_copula

from src.experiments.covid_exp import COVIDDataset

import logging
import datetime
from tabulate import tabulate

In [21]:
horizon = 10
with open("processed_data/covid_conformal_%d.pkl" % horizon, "rb") as f:
    train_dataset, calibration_dataset, test_dataset = pickle.load(f)
    
for i in range(3):

    with open("trained/200models_covid_daily_vanilla%d.pkl" % i, "rb") as f:
        model_list = pickle.load(f)
    covid_predictions = {"rnn_train_preds": model_list[1](train_dataset.X)[0],
                       "rnn_calib_preds": model_list[1](calibration_dataset.X)[0],
                       "rnn_test_preds": model_list[1](test_dataset.X)[0],
                       "lstm_train_preds": model_list[0].predict(train_dataset.X),
                       "lstm_calib_preds": model_list[0].predict(calibration_dataset.X),
                       "lstm_test_preds": model_list[0].predict(test_dataset.X)}

    #with open("./covid_data/covid_predictions_200epochs_%dhorizon_%d.pkl" % (horizon, i), "wb") as f:
        #pickle.dump(covid_predictions, f)

In [15]:
keys = ['dprnn', 'cfrnn', 'copula-rnn', 'cf-EncDec', 'copula-EncDec']


default_value = []  # You can set the default value if needed

cov_dict = {key: [] for key in keys}

area_dict = {key: [] for key in keys}
    


for i in range(3):
    #with open("trained/results_covid_daily_vanilla%d.pkl" % i, "rb") as f:
    with open("trained/results_particle5_run%d.pkl" % i, "rb") as f:
        results = pickle.load(f)
    
    for k in keys:
        cov_dict[k].append(torch.tensor(results[1][k]).mean().item())
        area_dict[k].append(torch.tensor(results[0][k]).mean().item())
    
cov_means = {key: [] for key in keys}
area_means = {key: [] for key in keys}  

cov_sds = {key: [] for key in keys}
area_sds = {key: [] for key in keys}  
    
for k in keys:
    cov_means[k] = torch.tensor(cov_dict[k]).mean().item()
    cov_sds[k] = torch.tensor(cov_dict[k]).var().pow(.5).item()
    area_means[k] = torch.tensor(area_dict[k]).mean().item()
    area_sds[k] = torch.tensor(area_dict[k]).var().pow(.5).item()

# Combine means and sds dictionaries into a single dictionary
combined_data = {
    method: (cov_means[method], cov_sds[method], area_means[method], area_sds[method])
    for method in means
}

# Create table headers
headers = ["Method", "Coverage", "Area", "Secondary Coverage", "Secondary Area"]

# Create table rows
table_data = [
    (method, "{:.4f} $\\pm$ {:.2f}".format(data[0], data[1]), "{:.4f} $\\pm$ {:.2f}".format(data[2], data[3]))
    for method, data in combined_data.items()
]

# Print the table in LaTeX format
print(tabulate(table_data, headers=headers, tablefmt="latex_raw"))

\begin{tabular}{lll}
\hline
 Method        & Coverage          & Area              \\
\hline
 dprnn         & 0.2057 $\pm$ 0.00 & 1.2376 $\pm$ 0.02 \\
 cfrnn         & 0.8451 $\pm$ 0.02 & 4.5444 $\pm$ 0.43 \\
 copula-rnn    & 0.7381 $\pm$ 0.01 & 4.0305 $\pm$ 0.41 \\
 cf-EncDec     & 0.8573 $\pm$ 0.03 & 5.0320 $\pm$ 0.80 \\
 copula-EncDec & 0.7381 $\pm$ 0.02 & 4.2340 $\pm$ 0.10 \\
\hline
\end{tabular}


In [59]:

datapaths = {"train": "nridata/loc_train_springs2_noise_0.05.npy",
             "valid": "nridata/loc_valid_springs2_noise_0.05.npy",
             "test": "nridata/loc_test_springs2_noise_0.05.npy"}



for i in range(3):
    particle_predictions = {}
    with open("trained/models_particle5_run%d.pkl" % i, "rb") as f:
        model_list = pickle.load(f)
    for k,v in datapaths.items():
        data = np.load(v)
        
        x = torch.tensor(data[:, :35, 0, :], dtype=torch.float)
        y = torch.tensor(data[:, 35:, 0, :], dtype=torch.float)
            
        particle_predictions["rnn_%s_preds" % k]= model_list[0](x)[0]
        particle_predictions["lstm_%s_preds" % k] = model_list[1].predict(x)
        
        with open("./particle_data/particle5_y_%s.pkl" % k, "wb") as f:
            pickle.dump(y, f)
    

    with open("./particle_data/particle5_predictions_%d.pkl" % i, "wb") as f:
        pickle.dump(particle_predictions, f)
        
    
datapaths = {'train': "nridata/loc_train_springs2_noise_0.01.npy",
             'valid': "nridata/loc_valid_springs2_noise_0.01.npy",
             'test': "nridata/loc_test_springs2_noise_0.01.npy"}


for i in range(3):
    particle_predictions = {}
    with open("trained/models_particle1_run%d.pkl" % i, "rb") as f:
        model_list = pickle.load(f)
    for k,v in datapaths.items():
        data = np.load(v)
        
        x = torch.tensor(data[:, :35, 0, :], dtype=torch.float)
        y = torch.tensor(data[:, 35:, 0, :], dtype=torch.float)
            
        particle_predictions["rnn_%s_preds" % k]= model_list[0](x)[0]
        particle_predictions["lstm_%s_preds" % k] = model_list[1].predict(x)
        
        with open("./particle_data/particle1_y_%s.pkl" % k, "wb") as f:
            pickle.dump(y, f)
        
    with open("./particle_data/particle1_predictions_%d.pkl" % i, "wb") as f:
        pickle.dump(particle_predictions, f)

In [58]:
particle_predictions.keys()

dict_keys(['rnn_train_preds', 'lstm_train_preds', 'rnn_valid_preds', 'lstm_valid_preds', 'rnn_test_preds', 'lstm_test_preds'])